# Executive Order Sentiment

In [3]:
import json
import os
from typing import LiteralString
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pydantic import BaseModel, Field

## Cleanup

In [ ]:
# load all the raw EOs do some cleanup
source_dir = 'data/executive_orders/raw/'
target_dir = 'data/executive_orders/cleaned/'
eo_paths = os.listdir(source_dir)
eo_paths.sort()

for eo_path in eo_paths[0:2000]:
    with open(source_dir + eo_path) as f:
        eo = json.load(f)

        # clean the byline by replacing \u2010 with -
        eo['president_byline'] = eo['president_byline'].replace('\u2010', '-')

        # content is an array of strings, flatmap strings with new lines into seperate strings
        flattened_content = []
        for s in eo['content']:
            flattened_content.extend(s.splitlines())
        eo["content"] = flattened_content

        # starting from the end, pick lines with less than 20 words up to the last 5 lines
        signature_lines = []
        for i in range(1, min(6, len(eo['content']))):
            cur_line = eo['content'][-i]
            if len(cur_line.split()) < 20:
                signature_lines.append(cur_line)
            else:
                break;
        
        if len(signature_lines) > 0:
            # reverse the signature lines
            signature_lines.reverse()
            # mask the signature lines
            masked_response = structured_llm.invoke({
                'input': json.dumps(signature_lines)
            })
            masked_lines = [] if masked_response is None else masked_response.lines

            # skip if the LLM had an obvious wrong solution
            if (len(masked_lines) == len(signature_lines)):
                # replace the signature lines with the masked lines
                eo['content'] = eo['content'][0:-len(signature_lines)] + masked_lines

        # write the cleaned eo to the target directory
        with open(target_dir + eo_path, 'w') as f:
            json.dump(eo, f, indent=4)

## Sentiment

https://arxiv.org/pdf/2312.06281

In [4]:
llm = ChatOllama(
    base_url="http://tr-pro.l.co:11434",
    model = "llama3.3",
    temperature = 0.0,
    num_predict = 9000,
    num_ctx = 4096
)

In [5]:
class EmotionScores(BaseModel):
    joy: int = Field(description="Joy conveyed on a scale of 1-10.")
    sadness: int = Field(description="sadness conveyed on a scale of 1-10.")
    fear: int = Field(description="fear conveyed on a scale of 1-10.")
    disgust: int = Field(description="disgust conveyed on a scale of 1-10.")
    surprise: int = Field(description="surprise conveyed on a scale of 1-10.")

class EmotionResponse(BaseModel):
    initial_scores: EmotionScores = Field(description="Initial emotion scores.")
    critique: str = Field(description="Critique of the initial scores.")
    revised_scores: EmotionScores = Field(description="Revised emotion scores.")


prompt = ChatPromptTemplate.from_template(
    '\n '.join([
        'Your task is to understand and score the emotions the president is conveying in the following executive order.',
        '- Does the president want the audience to feel joy, sadness, anger, fear, disgust, or surprise after reading the executive order? '
        '- For each emotion that should be conveyed, score the relative intensity on a scale of 1-10.',
        '- Critique your answer by thinking through it step by step.'
        '- Finally, give your revised scores.'
        '{text}',
    ])
)

structured_llm = prompt | llm.with_structured_output(EmotionResponse, include_raw=True)

In [ ]:
trump_text = '\n'.join([
    "By the authority vested in me as President by the Constitution and the laws of the United States of America, it is hereby ordered:",
    "Section 1. Purpose. It is the policy of my Administration to focus the executive branch's limited enforcement resources on regulations squarely authorized by constitutional Federal statutes, and to commence the deconstruction of the overbearing and burdensome administrative state. Ending Federal overreach and restoring the constitutional separation of powers is a priority of my Administration.",
    "Sec. 2. Rescinding Unlawful Regulations and Regulations That Undermine the National Interest. (a) Agency heads shall, in coordination with their DOGE Team Leads and the Director of the Office of Management and Budget, initiate a process to review all regulations subject to their sole or joint jurisdiction for consistency with law and Administration policy. Within 60 days of the date of this order, agency heads shall, in consultation with the Attorney General as appropriate, identify the following classes of regulations:",
    "(i) unconstitutional regulations and regulations that raise serious constitutional difficulties, such as exceeding the scope of the power vested in the Federal Government by the Constitution;",
    "(ii) regulations that are based on unlawful delegations of legislative power;",
    "(iii) regulations that are based on anything other than the best reading of the underlying statutory authority or prohibition;",
    "(iv) regulations that implicate matters of social, political, or economic significance that are not authorized by clear statutory authority;",
    "(v) regulations that impose significant costs upon private parties that are not outweighed by public benefits;",
    "(vi) regulations that harm the national interest by significantly and unjustifiably impeding technological innovation, infrastructure development, disaster response, inflation reduction, research and development, economic development, energy production, land use, and foreign policy objectives; and",
    "(vii) regulations that impose undue burdens on small business and impede private enterprise and entrepreneurship.",
    "(b) In conducting the review required by subsection (a) of this section, agencies shall prioritize review of those rules that satisfy the definition of \"significant regulatory action\" in Executive Order 12866 of September 30, 1993 (Regulatory Planning and Review), as amended.",
    "(c) Within 60 days of the date of this order, agency heads shall provide to the Administrator of the Office of Information and Regulatory Affairs (OIRA) within the Office of Management and Budget a list of all regulations identified by class as listed in subsection (a) of this section.",
    "(d) The Administrator of OIRA shall consult with agency heads to develop a Unified Regulatory Agenda that seeks to rescind or modify these regulations, as appropriate.",
    "Sec. 3. Enforcement Discretion to Ensure Lawful Governance. (a) Subject to their paramount obligation to discharge their legal obligations, protect public safety, and advance the national interest, agencies shall preserve their limited enforcement resources by generally de-prioritizing actions to enforce regulations that are based on anything other than the best reading of a statute and de-prioritizing actions to enforce regulations that go beyond the powers vested in the Federal Government by the Constitution.",
    "(b) Agency heads shall determine whether ongoing enforcement of any regulations identified in their regulatory review is compliant with law and Administration policy. To preserve resources and ensure lawful enforcement, agency heads, in consultation with the Director of the Office of Management and Budget, shall, on a case-by-case basis and as appropriate and consistent with applicable law, then direct the termination of all such enforcement proceedings that do not comply with the Constitution, laws, or Administration policy.",
    "Sec. 4. Promulgation of New Regulations. Agencies shall continue to follow the processes set out in Executive Order 12866 for submitting regulations for review by OIRA. Additionally, agency heads shall consult with their DOGE Team Leads and the Administrator of OIRA on potential new regulations as soon as practicable. In evaluating potential new regulations, agency heads, DOGE Team Leads, and the Administrator of OIRA shall consider, in addition to the factors set out in Executive Order 12866, the factors set out in section 2(a) of this order.",
    "Sec. 5. Implementation. The Director of the Office of Management and Budget shall issue implementation guidance, as appropriate.",
    "Sec. 6. Definitions. (a) \"Agency\" has the meaning given to it in 44 U.S.C. 3502, except it does not include the Executive Office of the President or its components.",
    "(b) \"Agency head\" shall mean the highest-ranking official of an agency, such as the Secretary, Administrator, Chairman, or Director.",
    "(c) \"DOGE Team Lead\" shall mean the leader of the DOGE Team at each agency as described in Executive Order 14158 of January 20, 2025 (Establishing and Implementing the President's \"Department of Government Efficiency\").",
    "(d) \"Enforcement action\" means all attempts, civil or criminal, by any agency to deprive a private party of life, liberty, or property, or in any way affect a private party's rights or obligations, regardless of the label the agency has historically placed on the action.",
    "(e) \"Regulation\" shall have the meaning given to \"regulatory action\" in section 3(e) of Executive Order 12866, and also includes any \"guidance document\" as defined in Executive Order 13422 of January 18, 2007 (Further Amendment to Executive Order 12866 on Regulatory Planning and Review).",
    "(f) \"Senior appointee\" means an individual appointed by the President, or performing the functions and duties of an office that requires appointment by the President, or a non-career member of the Senior Executive Service (or equivalent agency system).",
    "Sec. 7. Exemptions. Notwithstanding any other provision in this order, nothing in this order shall apply to:",
    "(a) any action related to a military, national security, homeland security, foreign affairs, or immigration-related function of the United States;",
    "(b) any matter pertaining to the executive branch's management of its employees; or",
    "(c) anything else exempted by the Director of the Office of Management and Budget.",
    "Sec. 8. Severability. If any provision of this order, or the application of any provision to any person or circumstance, is held to be invalid, the remainder of this order and the application of its provisions to any other persons or circumstances shall not be affected thereby.",
    "Sec. 9. General Provisions. (a) Nothing in this order shall be construed to impair or otherwise affect:",
    "(i) the authority granted by law to an executive department, agency, or the head thereof; or",
    "(ii) the functions of the Director of the Office of Management and Budget relating to budgetary, administrative, or legislative proposals.",
    "(b) This order shall be implemented consistent with applicable law and subject to the availability of appropriations.",
    "(c) This order is not intended to, and does not, create any right or benefit, substantive or procedural, enforceable at law or in equity by any party against the United States, its departments, agencies, or entities, its officers, employees, or agents, or any other person.",
    "[NAME]",
    "The White House,",
    "February 19, 2025."
])

biden_text = '\n'.join([
    "By the authority vested in me as President by the Constitution and the laws of the United States of America, including the Federal Property and Administrative Services Act, 40 U.S.C. 101 et seq., and in order to promote economy and efficiency in procurement by contracting with sources that adequately compensate their workers, it is hereby ordered as follows:",
    "Section 1. Policy. This order promotes economy and efficiency in Federal procurement by increasing the hourly minimum wage paid by the parties that contract with the Federal Government to $15.00 for those workers working on or in connection with a Federal Government contract as described in section 8 of this order. Raising the minimum wage enhances worker productivity and generates higher-quality work by boosting workers' health, morale, and effort; reducing absenteeism and turnover; and lowering supervisory and training costs. Accordingly, ensuring that Federal contractors pay their workers an hourly wage of at least $15.00 will bolster economy and efficiency in Federal procurement.",
    "Sec. 2. Increasing the Minimum Wage for Federal Contractors and Subcontractors. (a) Executive departments and agencies, including independent establishments subject to the Federal Property and Administrative Services Act, 40 U.S.C. 102(4)(A), (5) (agencies), shall, to the extent permitted by law, ensure that contracts and contract-like instruments (as defined in regulations issued pursuant to section 4(a) of this order and as described in section 8(a) of this order) include a clause that the contractor and any covered subcontractors (as defined in regulations issued pursuant to section 4(a) of this order) shall incorporate into lower-tier subcontracts. This clause shall specify that, as a condition of payment, the minimum wage to be paid to workers employed in the performance of the contract or any covered subcontract thereunder, including workers whose wages are calculated pursuant to special certificates issued under section 14(c) of the Fair Labor Standards Act of 1938 (29 U.S.C. 214(c)), shall be at least:",
    "(i) $15.00 per hour, beginning January 30, 2022; and",
    "(ii) beginning January 1, 2023, and annually thereafter, an amount determined by the Secretary of Labor (Secretary). The amount shall be published by the Secretary at least 90 days before such new minimum wage is to take effect and shall be:",
    "(A) not less than the amount in effect on the date of such determination;",
    "(B) increased from such amount by the annual percentage increase in the Consumer Price Index for Urban Wage Earners and Clerical Workers (United States city average, all items, not seasonally adjusted), or its successor publication, as determined by the Bureau of Labor Statistics; and",
    "(C) rounded to the nearest multiple of $0.05.",
    "(b) In calculating the annual percentage increase in the Consumer Price Index for purposes of subsection (a)(ii)(B) of this section, the Secretary shall compare such Consumer Price Index for the most recent month, quarter, or year available (as selected by the Secretary prior to the first year for which a minimum wage is in effect pursuant to subsection (a)(ii)(B) of this section) with the Consumer Price Index for the same month in the preceding year, the same quarter in the preceding year, or the preceding year, respectively. (c) Nothing in this order shall excuse noncompliance with any applicable Federal or State prevailing wage law, or any applicable law or municipal ordinance establishing a minimum wage higher than the minimum wage established under this order.",
    "Sec. 3. Application to Tipped Workers. (a) For workers covered under section 2 of this order who are tipped employees pursuant to section 3(t) of the Fair Labor Standards Act of 1938 (29 U.S.C. 203(t)), the cash wage that must be paid by an employer to such workers shall be at least:",
    "(i) $10.50 per hour, beginning January 30, 2022;",
    "(ii) beginning January 1, 2023, 85 percent of the wage in effect under section 2 of this order, rounded to the nearest multiple of $0.05; and",
    "(iii) beginning January 1, 2024, and for each subsequent year, 100 percent of the wage in effect under section 2 of this order.",
    "(b) Where workers do not receive a sufficient additional amount on account of tips, when combined with the hourly cash wage paid by the employer, such that their wages are equal to the minimum wage under section 2 of this order, the cash wage paid by the employer, as set forth in this section for those workers, shall be increased such that their wages equal the minimum wage under section 2 of this order. Consistent with applicable law, if the wage required to be paid under the Service Contract Act, 41 U.S.C. 6701 et seq., or any other applicable law or regulation is higher than the wage required under section 2 of this order, the employer shall pay additional cash wages sufficient to meet the highest wage required to be paid.",
    "Sec. 4. Regulations and Implementation. (a) The Secretary shall, consistent with applicable law, issue regulations by November 24, 2021, to implement the requirements of this order. Such regulations shall include both definitions of relevant terms and, as appropriate, exclusions from the requirements of this order. Within 60 days of the Secretary issuing such regulations, the Federal Acquisition Regulatory Council, to the extent permitted by law, shall amend the Federal Acquisition Regulation to provide for inclusion in Federal procurement solicitations, contracts, and contract-like instruments subject to this order the clause described in section 2(a) of this order.",
    "(b) Within 60 days of the Secretary issuing regulations pursuant to subsection (a) of this section, agencies shall take steps, to the extent permitted by law, to exercise any applicable authority to ensure that contracts and contract-like instruments as described in sections 8(a)(i)(C) and (D) of this order, entered into on or after January 30, 2022, consistent with the effective date of such agency action, comply with the requirements set forth in sections 2 and 3 of this order.",
    "(c) Any regulations issued pursuant to this section should, to the extent practicable, incorporate existing definitions, principles, procedures, remedies, and enforcement processes under the Fair Labor Standards Act of 1938, 29 U.S.C. 201 et seq.; the Service Contract Act, 41 U.S.C. 6701 et seq.; the Davis-Bacon Act, 40 U.S.C. 3141 et seq.; Executive Order 13658 of February 12, 2014 (Establishing a Minimum Wage for Contractors); and regulations issued to implement that order.",
    "Sec. 5. Enforcement. (a) The Secretary shall have the authority for investigating potential violations of and obtaining compliance with this order.",
    "(b) This order creates no rights under the Contract Disputes Act, 41 U.S.C. 7101 et seq., and disputes regarding whether a contractor has paid the wages prescribed by this order, as appropriate and consistent with applicable law, shall be disposed of only as provided by the Secretary in regulations issued pursuant to this order.",
    "Sec. 6. Revocation of Certain Presidential Actions. Executive Order 13838 of May 25, 2018 (Exemption From Executive Order 13658 for Recreational Services on Federal Lands), is revoked as of January 30, 2022. Executive Order 13658 of February 12, 2014 (Establishing a Minimum Wage for Contractors), is superseded, as of January 30, 2022, to the extent it is inconsistent with this order.",
    "Sec. 7. Severability. If any provision of this order, or the application of any provision of this order to any person or circumstance, is held to be invalid, the remainder of this order and its application to any other person or circumstance shall not be affected thereby.",
    "Sec. 8. Applicability. (a) This order shall apply to any new contract; new contract-like instrument; new solicitation; extension or renewal of an existing contract or contract-like instrument; and exercise of an option on an existing contract or contract-like instrument, if (i):",
    "(A) it is a procurement contract or contract-like instrument for services or construction;",
    "(B) it is a contract or contract-like instrument for services covered by the Service Contract Act;",
    "(C) it is a contract or contract-like instrument for concessions, including any concessions contract excluded by Department of Labor regulations at 29 C.F.R. 4.133(b); or",
    "(D) it is a contract or contract-like instrument entered into with the Federal Government in connection with Federal property or lands and related to offering services for Federal employees, their dependents, or the general public; and",
    "(ii) the wages of workers under such contract or contract-like instrument are governed by the Fair Labor Standards Act, the Service Contract Act, or the Davis-Bacon Act.",
    "(b) For contracts or contract-like instruments covered by the Service Contract Act or the Davis-Bacon Act, this order shall apply only to contracts or contract-like instruments at the thresholds specified in those statutes. Where workers' wages are governed by the Fair Labor Standards Act of 1938, this order shall apply only to procurement contracts or contract-like instruments that exceed the micro-purchase threshold, as defined in 41 U.S.C. 1902(a), unless expressly made subject to this order pursuant to regulations or actions taken under section 4 of this order.",
    "(c) This order shall not apply to grants; contracts, contract-like instruments, or agreements with Indian Tribes under the Indian Self-Determination and Education Assistance Act (Public Law 93-638), as amended; or any contracts or contract-like instruments expressly excluded by the regulations issued pursuant to section 4(a) of this order.",
    "Sec. 9. Effective Date. (a) This order is effective immediately and shall apply to new contracts; new contract-like instruments; new solicitations; extensions or renewals of existing contracts or contract-like instruments; and exercises of options on existing contracts or contract-like instruments, as described in section 8(a) in this order, where the relevant contract or contract-like instrument will be entered into, the relevant contract or contract-like instrument will be extended or renewed, or the relevant option will be exercised, on or after:",
    "(i) January 30, 2022, consistent with the effective date for the action taken by the Federal Acquisition Regulatory Council pursuant to section 4(a) of this order; or",
    "(ii) for contracts where an agency action is taken pursuant to section 4(b) of this order, January 30, 2022, consistent with the effective date for such action.",
    "(b) As an exception to subsection (a) of this section, where agencies have issued a solicitation before the effective date for the relevant action taken pursuant to section 4 of this order and entered into a new contract or contract-like instrument resulting from such solicitation within 60 days of such effective date, such agencies are strongly encouraged but not required to ensure that the minimum wages specified in sections 2 and 3 of this order are paid in the new contract or contract-like instrument. But if that contract or contract-like instrument is subsequently extended or renewed, or an option is subsequently exercised under that contract or contract-like instrument, the minimum wages specified in sections 2 and 3 of this order shall apply to that extension, renewal, or option.",
    "(c) For all existing contracts and contract-like instruments, solicitations issued between the date of this order and the effective dates set forth in this section, and contracts and contract-like instruments entered into between the date of this order and the effective dates set forth in this section, agencies are strongly encouraged, to the extent permitted by law, to ensure that the hourly wages paid under such contracts or contract-like instruments are consistent with the minimum wages specified in sections 2 and 3 of this order.",
    "Sec. 10. General Provisions. (a) Nothing in this order shall be construed to impair or otherwise affect:",
    "(i) the authority granted by law to an executive department or agency, or the head thereof; or",
    "(ii) the functions of the Director of the Office of Management and Budget relating to budgetary, administrative, or legislative proposals.",
    "(b) This order shall be implemented consistent with applicable law and subject to the availability of appropriations.",
    "(c) This order is not intended to, and does not, create any right or benefit, substantive or procedural, enforceable at law or in equity by any party against the United States, its departments, agencies, or entities, its officers, employees, or agents, or any other person.",
    "[NAME]",
    "The White House,",
    "April 27, 2021."
])

In [ ]:
structured_llm.invoke(
    {
        'text': 
    }
)

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.3', 'created_at': '2025-03-07T03:23:23.702673356Z', 'done': True, 'done_reason': 'stop', 'total_duration': 24792135457, 'load_duration': 7216955976, 'prompt_eval_count': 1718, 'prompt_eval_duration': 3325000000, 'eval_count': 214, 'eval_duration': 14248000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a01d160f-d992-4e3f-931c-3fbbd8ba64b1-0', tool_calls=[{'name': 'EmotionResponse', 'args': {'critique': "The executive order is written in a formal and objective tone, but the language used suggests a sense of frustration and dissatisfaction with the current state of regulations. The president's emphasis on 'deconstruction of the overbearing and burdensome administrative state' and 'ending Federal overreach' implies a strong negative emotion towards the existing regulatory framework. The order's focus on rescinding unlawful regulations and promoting economic

In [7]:
qwen_model = ChatOllama(
    base_url="http://tr-pro.l.co:11434",
    model = "qwen2.5",
    temperature = 0.0,
    num_predict = 9000,
    num_ctx = 4096
)

structured_qwen = prompt | qwen_model.with_structured_output(EmotionResponse, include_raw=True)

In [8]:
structured_qwen.invoke(
    {
        'text': '\n'.join([
        "By the authority vested in me as President by the Constitution and the laws of the United States of America, it is hereby ordered:",
        "Section 1. Purpose. It is the policy of my Administration to focus the executive branch's limited enforcement resources on regulations squarely authorized by constitutional Federal statutes, and to commence the deconstruction of the overbearing and burdensome administrative state. Ending Federal overreach and restoring the constitutional separation of powers is a priority of my Administration.",
        "Sec. 2. Rescinding Unlawful Regulations and Regulations That Undermine the National Interest. (a) Agency heads shall, in coordination with their DOGE Team Leads and the Director of the Office of Management and Budget, initiate a process to review all regulations subject to their sole or joint jurisdiction for consistency with law and Administration policy. Within 60 days of the date of this order, agency heads shall, in consultation with the Attorney General as appropriate, identify the following classes of regulations:",
        "(i) unconstitutional regulations and regulations that raise serious constitutional difficulties, such as exceeding the scope of the power vested in the Federal Government by the Constitution;",
        "(ii) regulations that are based on unlawful delegations of legislative power;",
        "(iii) regulations that are based on anything other than the best reading of the underlying statutory authority or prohibition;",
        "(iv) regulations that implicate matters of social, political, or economic significance that are not authorized by clear statutory authority;",
        "(v) regulations that impose significant costs upon private parties that are not outweighed by public benefits;",
        "(vi) regulations that harm the national interest by significantly and unjustifiably impeding technological innovation, infrastructure development, disaster response, inflation reduction, research and development, economic development, energy production, land use, and foreign policy objectives; and",
        "(vii) regulations that impose undue burdens on small business and impede private enterprise and entrepreneurship.",
        "(b) In conducting the review required by subsection (a) of this section, agencies shall prioritize review of those rules that satisfy the definition of \"significant regulatory action\" in Executive Order 12866 of September 30, 1993 (Regulatory Planning and Review), as amended.",
        "(c) Within 60 days of the date of this order, agency heads shall provide to the Administrator of the Office of Information and Regulatory Affairs (OIRA) within the Office of Management and Budget a list of all regulations identified by class as listed in subsection (a) of this section.",
        "(d) The Administrator of OIRA shall consult with agency heads to develop a Unified Regulatory Agenda that seeks to rescind or modify these regulations, as appropriate.",
        "Sec. 3. Enforcement Discretion to Ensure Lawful Governance. (a) Subject to their paramount obligation to discharge their legal obligations, protect public safety, and advance the national interest, agencies shall preserve their limited enforcement resources by generally de-prioritizing actions to enforce regulations that are based on anything other than the best reading of a statute and de-prioritizing actions to enforce regulations that go beyond the powers vested in the Federal Government by the Constitution.",
        "(b) Agency heads shall determine whether ongoing enforcement of any regulations identified in their regulatory review is compliant with law and Administration policy. To preserve resources and ensure lawful enforcement, agency heads, in consultation with the Director of the Office of Management and Budget, shall, on a case-by-case basis and as appropriate and consistent with applicable law, then direct the termination of all such enforcement proceedings that do not comply with the Constitution, laws, or Administration policy.",
        "Sec. 4. Promulgation of New Regulations. Agencies shall continue to follow the processes set out in Executive Order 12866 for submitting regulations for review by OIRA. Additionally, agency heads shall consult with their DOGE Team Leads and the Administrator of OIRA on potential new regulations as soon as practicable. In evaluating potential new regulations, agency heads, DOGE Team Leads, and the Administrator of OIRA shall consider, in addition to the factors set out in Executive Order 12866, the factors set out in section 2(a) of this order.",
        "Sec. 5. Implementation. The Director of the Office of Management and Budget shall issue implementation guidance, as appropriate.",
        "Sec. 6. Definitions. (a) \"Agency\" has the meaning given to it in 44 U.S.C. 3502, except it does not include the Executive Office of the President or its components.",
        "(b) \"Agency head\" shall mean the highest-ranking official of an agency, such as the Secretary, Administrator, Chairman, or Director.",
        "(c) \"DOGE Team Lead\" shall mean the leader of the DOGE Team at each agency as described in Executive Order 14158 of January 20, 2025 (Establishing and Implementing the President's \"Department of Government Efficiency\").",
        "(d) \"Enforcement action\" means all attempts, civil or criminal, by any agency to deprive a private party of life, liberty, or property, or in any way affect a private party's rights or obligations, regardless of the label the agency has historically placed on the action.",
        "(e) \"Regulation\" shall have the meaning given to \"regulatory action\" in section 3(e) of Executive Order 12866, and also includes any \"guidance document\" as defined in Executive Order 13422 of January 18, 2007 (Further Amendment to Executive Order 12866 on Regulatory Planning and Review).",
        "(f) \"Senior appointee\" means an individual appointed by the President, or performing the functions and duties of an office that requires appointment by the President, or a non-career member of the Senior Executive Service (or equivalent agency system).",
        "Sec. 7. Exemptions. Notwithstanding any other provision in this order, nothing in this order shall apply to:",
        "(a) any action related to a military, national security, homeland security, foreign affairs, or immigration-related function of the United States;",
        "(b) any matter pertaining to the executive branch's management of its employees; or",
        "(c) anything else exempted by the Director of the Office of Management and Budget.",
        "Sec. 8. Severability. If any provision of this order, or the application of any provision to any person or circumstance, is held to be invalid, the remainder of this order and the application of its provisions to any other persons or circumstances shall not be affected thereby.",
        "Sec. 9. General Provisions. (a) Nothing in this order shall be construed to impair or otherwise affect:",
        "(i) the authority granted by law to an executive department, agency, or the head thereof; or",
        "(ii) the functions of the Director of the Office of Management and Budget relating to budgetary, administrative, or legislative proposals.",
        "(b) This order shall be implemented consistent with applicable law and subject to the availability of appropriations.",
        "(c) This order is not intended to, and does not, create any right or benefit, substantive or procedural, enforceable at law or in equity by any party against the United States, its departments, agencies, or entities, its officers, employees, or agents, or any other person.",
        "DONALD J. TRUMP",
        "The White House,",
        "February 19, 2025."
        ])
    }
)

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-03-07T03:23:50.945367305Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11572552230, 'load_duration': 3307211309, 'prompt_eval_count': 1756, 'prompt_eval_duration': 507000000, 'eval_count': 764, 'eval_duration': 7384000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-0f02da5a-d0b5-427e-b03c-dd62e6281495-0', tool_calls=[{'name': 'EmotionResponse', 'args': {'critique': 'The initial scores were based on a careful analysis of the text. The order is focused on policy changes and does not evoke strong emotions like joy or sadness. Anger and fear are present but not at high levels. Disgust is more about dissatisfaction with current regulations, while surprise comes from the detailed nature of the order.', 'initial_scores': {'anger': 3, 'disgust': 4, 'fear': 2, 'joy': 1, 'sadness': 1, 'surprise': 7}, 'revised_scores': {'anger': 3

In [9]:
qwq_model = ChatOllama(
    model = "qwq",
    temperature = 0.0,
    num_predict = 9000,
    num_ctx = 4096
)

structured_qwq = prompt | qwq_model.with_structured_output(EmotionResponse, include_raw=True)

In [10]:
structured_qwq.invoke(
    {
        'text': '\n'.join([
        "By the authority vested in me as President by the Constitution and the laws of the United States of America, it is hereby ordered:",
        "Section 1. Purpose. It is the policy of my Administration to focus the executive branch's limited enforcement resources on regulations squarely authorized by constitutional Federal statutes, and to commence the deconstruction of the overbearing and burdensome administrative state. Ending Federal overreach and restoring the constitutional separation of powers is a priority of my Administration.",
        "Sec. 2. Rescinding Unlawful Regulations and Regulations That Undermine the National Interest. (a) Agency heads shall, in coordination with their DOGE Team Leads and the Director of the Office of Management and Budget, initiate a process to review all regulations subject to their sole or joint jurisdiction for consistency with law and Administration policy. Within 60 days of the date of this order, agency heads shall, in consultation with the Attorney General as appropriate, identify the following classes of regulations:",
        "(i) unconstitutional regulations and regulations that raise serious constitutional difficulties, such as exceeding the scope of the power vested in the Federal Government by the Constitution;",
        "(ii) regulations that are based on unlawful delegations of legislative power;",
        "(iii) regulations that are based on anything other than the best reading of the underlying statutory authority or prohibition;",
        "(iv) regulations that implicate matters of social, political, or economic significance that are not authorized by clear statutory authority;",
        "(v) regulations that impose significant costs upon private parties that are not outweighed by public benefits;",
        "(vi) regulations that harm the national interest by significantly and unjustifiably impeding technological innovation, infrastructure development, disaster response, inflation reduction, research and development, economic development, energy production, land use, and foreign policy objectives; and",
        "(vii) regulations that impose undue burdens on small business and impede private enterprise and entrepreneurship.",
        "(b) In conducting the review required by subsection (a) of this section, agencies shall prioritize review of those rules that satisfy the definition of \"significant regulatory action\" in Executive Order 12866 of September 30, 1993 (Regulatory Planning and Review), as amended.",
        "(c) Within 60 days of the date of this order, agency heads shall provide to the Administrator of the Office of Information and Regulatory Affairs (OIRA) within the Office of Management and Budget a list of all regulations identified by class as listed in subsection (a) of this section.",
        "(d) The Administrator of OIRA shall consult with agency heads to develop a Unified Regulatory Agenda that seeks to rescind or modify these regulations, as appropriate.",
        "Sec. 3. Enforcement Discretion to Ensure Lawful Governance. (a) Subject to their paramount obligation to discharge their legal obligations, protect public safety, and advance the national interest, agencies shall preserve their limited enforcement resources by generally de-prioritizing actions to enforce regulations that are based on anything other than the best reading of a statute and de-prioritizing actions to enforce regulations that go beyond the powers vested in the Federal Government by the Constitution.",
        "(b) Agency heads shall determine whether ongoing enforcement of any regulations identified in their regulatory review is compliant with law and Administration policy. To preserve resources and ensure lawful enforcement, agency heads, in consultation with the Director of the Office of Management and Budget, shall, on a case-by-case basis and as appropriate and consistent with applicable law, then direct the termination of all such enforcement proceedings that do not comply with the Constitution, laws, or Administration policy.",
        "Sec. 4. Promulgation of New Regulations. Agencies shall continue to follow the processes set out in Executive Order 12866 for submitting regulations for review by OIRA. Additionally, agency heads shall consult with their DOGE Team Leads and the Administrator of OIRA on potential new regulations as soon as practicable. In evaluating potential new regulations, agency heads, DOGE Team Leads, and the Administrator of OIRA shall consider, in addition to the factors set out in Executive Order 12866, the factors set out in section 2(a) of this order.",
        "Sec. 5. Implementation. The Director of the Office of Management and Budget shall issue implementation guidance, as appropriate.",
        "Sec. 6. Definitions. (a) \"Agency\" has the meaning given to it in 44 U.S.C. 3502, except it does not include the Executive Office of the President or its components.",
        "(b) \"Agency head\" shall mean the highest-ranking official of an agency, such as the Secretary, Administrator, Chairman, or Director.",
        "(c) \"DOGE Team Lead\" shall mean the leader of the DOGE Team at each agency as described in Executive Order 14158 of January 20, 2025 (Establishing and Implementing the President's \"Department of Government Efficiency\").",
        "(d) \"Enforcement action\" means all attempts, civil or criminal, by any agency to deprive a private party of life, liberty, or property, or in any way affect a private party's rights or obligations, regardless of the label the agency has historically placed on the action.",
        "(e) \"Regulation\" shall have the meaning given to \"regulatory action\" in section 3(e) of Executive Order 12866, and also includes any \"guidance document\" as defined in Executive Order 13422 of January 18, 2007 (Further Amendment to Executive Order 12866 on Regulatory Planning and Review).",
        "(f) \"Senior appointee\" means an individual appointed by the President, or performing the functions and duties of an office that requires appointment by the President, or a non-career member of the Senior Executive Service (or equivalent agency system).",
        "Sec. 7. Exemptions. Notwithstanding any other provision in this order, nothing in this order shall apply to:",
        "(a) any action related to a military, national security, homeland security, foreign affairs, or immigration-related function of the United States;",
        "(b) any matter pertaining to the executive branch's management of its employees; or",
        "(c) anything else exempted by the Director of the Office of Management and Budget.",
        "Sec. 8. Severability. If any provision of this order, or the application of any provision to any person or circumstance, is held to be invalid, the remainder of this order and the application of its provisions to any other persons or circumstances shall not be affected thereby.",
        "Sec. 9. General Provisions. (a) Nothing in this order shall be construed to impair or otherwise affect:",
        "(i) the authority granted by law to an executive department, agency, or the head thereof; or",
        "(ii) the functions of the Director of the Office of Management and Budget relating to budgetary, administrative, or legislative proposals.",
        "(b) This order shall be implemented consistent with applicable law and subject to the availability of appropriations.",
        "(c) This order is not intended to, and does not, create any right or benefit, substantive or procedural, enforceable at law or in equity by any party against the United States, its departments, agencies, or entities, its officers, employees, or agents, or any other person.",
        "DONALD J. TRUMP",
        "The White House,",
        "February 19, 2025."
        ])
    }
)

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwq', 'created_at': '2025-03-07T03:25:16.687851146Z', 'done': True, 'done_reason': 'stop', 'total_duration': 28104807923, 'load_duration': 3212096126, 'prompt_eval_count': 1740, 'prompt_eval_duration': 1766000000, 'eval_count': 693, 'eval_duration': 23125000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a7a28d0c-6d67-4a05-be18-3574c2e49975-0', tool_calls=[{'name': 'EmotionResponse', 'args': {'critique': "The executive order emphasizes strong opposition to existing regulations, using terms like 'overbearing,' 'unlawful,' and 'burdensome.' Anger (initial 7) may reflect frustration with prior policies but could be tempered by the formal tone. Fear (5) is moderate as it focuses on systemic risks rather than direct threats. Disgust (6) aligns with disapproval of administrative excesses, while surprise (3) is low due to expected policy shifts. Revised scores adjust 